# TensorFlow FL for Bug Prediction - Improved Version

## Key Improvements:
1. **Better Model**: Proper feature normalization, optimized architecture
2. **Effective Poisoning Attacks**:
   - Model Poisoning: Sign-flip attack (negate weights)
   - Data Poisoning: Strategic label flipping

In [ ]:
!pip install tensorflow scikit-learn -q
print("Done!")

In [ ]:
import json
import time
import copy
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from typing import Dict, List
from dataclasses import dataclass, field, asdict
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
from google.colab import files
print("Upload dataset_pairs_1_.json")
uploaded = files.upload()
dataset_path = list(uploaded.keys())[0]

## 2. Enhanced Feature Extraction

In [ ]:
def extract_code_features(code: str) -> np.ndarray:
    """
    Extract discriminative features that differ between buggy and fixed code.
    """
    lines = code.split('\n')
    non_empty_lines = [l for l in lines if l.strip()]
    
    # Basic metrics
    total_chars = len(code)
    num_lines = len(lines)
    num_non_empty = len(non_empty_lines)
    avg_line_len = np.mean([len(l) for l in lines]) if lines else 0
    max_line_len = max([len(l) for l in lines]) if lines else 0
    
    # Code structure
    num_functions = code.count('def ')
    num_classes = code.count('class ')
    num_imports = code.count('import ')
    num_returns = code.count('return ')
    
    # Control flow
    num_if = code.count('if ') + code.count('elif ')
    num_else = code.count('else:')
    num_for = code.count('for ')
    num_while = code.count('while ')
    num_try = code.count('try:')
    num_except = code.count('except')
    num_finally = code.count('finally:')
    num_with = code.count('with ')
    
    # Operators (bug-prone)
    num_eq = code.count(' == ')
    num_neq = code.count(' != ')
    num_lt = code.count(' < ') + code.count(' <= ')
    num_gt = code.count(' > ') + code.count(' >= ')
    num_and = code.count(' and ')
    num_or = code.count(' or ')
    num_not = code.count(' not ') + code.count('not(')
    
    # Assignment and arithmetic
    num_assign = code.count(' = ') - num_eq - num_neq
    num_plus_eq = code.count(' += ')
    num_minus_eq = code.count(' -= ')
    num_arithmetic = code.count(' + ') + code.count(' - ') + code.count(' * ') + code.count(' / ')
    num_modulo = code.count(' % ')
    
    # Data structures
    num_list_access = code.count('[') 
    num_dict_access = code.count('{')
    num_tuple = code.count('(')
    
    # Common operations
    num_append = code.count('.append(')
    num_extend = code.count('.extend(')
    num_pop = code.count('.pop(')
    num_remove = code.count('.remove(')
    num_len = code.count('len(')
    num_range = code.count('range(')
    num_enumerate = code.count('enumerate(')
    num_zip = code.count('zip(')
    
    # String operations
    num_str_format = code.count('.format(') + code.count('f"') + code.count("f'")
    num_split = code.count('.split(')
    num_join = code.count('.join(')
    num_strip = code.count('.strip(') + code.count('.rstrip(') + code.count('.lstrip(')
    
    # Type operations
    num_int = code.count('int(')
    num_float = code.count('float(')
    num_str = code.count('str(')
    num_list = code.count('list(')
    num_dict = code.count('dict(')
    
    # Error-prone patterns
    num_none = code.count('None')
    num_true = code.count('True')
    num_false = code.count('False')
    num_is = code.count(' is ')
    num_in = code.count(' in ')
    
    # Indentation (complexity)
    indent_levels = [len(l) - len(l.lstrip()) for l in non_empty_lines] if non_empty_lines else [0]
    max_indent = max(indent_levels)
    avg_indent = np.mean(indent_levels)
    
    # Comments
    num_comments = code.count('#')
    num_docstrings = code.count('"""') + code.count("'''")
    
    features = np.array([
        total_chars, num_lines, num_non_empty, avg_line_len, max_line_len,
        num_functions, num_classes, num_imports, num_returns,
        num_if, num_else, num_for, num_while, num_try, num_except, num_finally, num_with,
        num_eq, num_neq, num_lt, num_gt, num_and, num_or, num_not,
        num_assign, num_plus_eq, num_minus_eq, num_arithmetic, num_modulo,
        num_list_access, num_dict_access, num_tuple,
        num_append, num_extend, num_pop, num_remove, num_len, num_range, num_enumerate, num_zip,
        num_str_format, num_split, num_join, num_strip,
        num_int, num_float, num_str, num_list, num_dict,
        num_none, num_true, num_false, num_is, num_in,
        max_indent, avg_indent, num_comments, num_docstrings
    ], dtype=np.float32)
    
    return features

print(f"Number of features: {len(extract_code_features('test'))}")

In [ ]:
# Load and process data
with open(dataset_path, 'r') as f:
    data = json.load(f)

print(f"Loaded {len(data)} code pairs")

# Extract features
features_list = []
labels_list = []

for pair in data:
    # Buggy code -> label 1
    features_list.append(extract_code_features(pair['buggy']))
    labels_list.append(1)
    
    # Fixed code -> label 0  
    features_list.append(extract_code_features(pair['fixed']))
    labels_list.append(0)

X = np.array(features_list)
y = np.array(labels_list)

print(f"Total samples: {len(y)}")
print(f"Features shape: {X.shape}")
print(f"Class distribution: Buggy={sum(y)}, Fixed={len(y)-sum(y)}")

In [ ]:
# Normalize features (CRITICAL for good performance)
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

print(f"Normalized feature range: [{X_normalized.min():.2f}, {X_normalized.max():.2f}]")
print(f"Mean: {X_normalized.mean():.4f}, Std: {X_normalized.std():.4f}")

## 3. Model Definition

In [ ]:
def create_model(input_dim):
    """
    Create a well-tuned model for bug prediction.
    """
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        
        # First hidden layer
        layers.Dense(128, kernel_initializer='he_normal'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.3),
        
        # Second hidden layer
        layers.Dense(64, kernel_initializer='he_normal'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.3),
        
        # Third hidden layer
        layers.Dense(32, kernel_initializer='he_normal'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        
        # Output
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Test
test_model = create_model(X_normalized.shape[1])
test_model.summary()

In [ ]:
# Verify model can learn (centralized training test)
print("Testing centralized training to verify model quality...\n")

X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.2, random_state=42, stratify=y
)

test_model = create_model(X_normalized.shape[1])
test_model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = test_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    verbose=1
)

y_pred = (test_model.predict(X_test, verbose=0) > 0.5).astype(int).flatten()
print(f"\n=== Centralized Performance ===")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred):.4f}")

## 4. Metrics & Utilities

In [ ]:
@dataclass
class Metrics:
    accuracy: float = 0.0
    precision: float = 0.0
    recall: float = 0.0
    f1_score: float = 0.0
    loss: float = 0.0
    
    def to_dict(self):
        return asdict(self)

@dataclass
class ExpResult:
    name: str
    metrics: Metrics
    time: float = 0.0
    comm_bytes: float = 0.0
    history: List = field(default_factory=list)
    
    def to_dict(self):
        return {
            'experiment_name': self.name,
            'metrics': self.metrics.to_dict(),
            'training_time': self.time,
            'communication_bytes': self.comm_bytes,
            'round_metrics': self.history
        }

def calc_metrics(y_true, y_pred, loss=0.0):
    return Metrics(
        accuracy=accuracy_score(y_true, y_pred),
        precision=precision_score(y_true, y_pred, zero_division=0),
        recall=recall_score(y_true, y_pred, zero_division=0),
        f1_score=f1_score(y_true, y_pred, zero_division=0),
        loss=loss
    )

def model_size(model):
    return sum(w.nbytes for w in model.get_weights())

## 5. Poisoning Attacks

In [ ]:
class Attacks:
    """
    Realistic poisoning attacks:
    
    1. Model Poisoning (Sign-Flip): 
       - Multiply weights by negative factor
       - Pushes model away from optimal
    
    2. Data Poisoning (Label Flip):
       - Flip labels on malicious clients
       - Model learns wrong patterns
    """
    
    @staticmethod
    def model_poison(weights, scale=-2.0):
        """
        Sign-flip attack with amplification.
        scale=-2.0 means flip sign and double magnitude.
        """
        return [w * scale for w in weights]
    
    @staticmethod
    def data_poison(labels):
        """Flip all labels: 0->1, 1->0"""
        return 1.0 - labels

print("Attacks defined:")
print("  - Model Poisoning: Sign-flip with 2x amplification")
print("  - Data Poisoning: Label flipping")

## 6. Federated Learning

In [ ]:
class Client:
    def __init__(self, cid, features, labels, input_dim, malicious=False, attack='none'):
        self.cid = cid
        self.malicious = malicious
        self.attack = attack
        
        # Data
        self.X = features.copy()
        self.y = Attacks.data_poison(labels.copy()) if (malicious and attack == 'data') else labels.copy()
        self.n_samples = len(self.y)
        
        # Model
        self.model = create_model(input_dim)
        self.model.compile(
            optimizer=keras.optimizers.Adam(0.001),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
    
    def set_weights(self, w):
        self.model.set_weights(w)
    
    def get_weights(self):
        w = self.model.get_weights()
        if self.malicious and self.attack == 'model':
            w = Attacks.model_poison(w)
        return w
    
    def train(self, epochs=5):
        h = self.model.fit(self.X, self.y, epochs=epochs, batch_size=16, verbose=0)
        return h.history['loss'][-1]


class Server:
    def __init__(self, input_dim):
        self.model = create_model(input_dim)
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    def get_weights(self):
        return self.model.get_weights()
    
    def aggregate(self, client_weights, client_samples):
        """FedAvg"""
        total = sum(client_samples)
        agg = [np.zeros_like(w) for w in client_weights[0]]
        for w, n in zip(client_weights, client_samples):
            for i in range(len(w)):
                agg[i] += w[i] * (n / total)
        self.model.set_weights(agg)
    
    def evaluate(self, X, y):
        pred = (self.model.predict(X, verbose=0) > 0.5).astype(int).flatten()
        loss = self.model.evaluate(X, y, verbose=0)[0]
        return pred, loss

In [ ]:
def run_experiment(
    client_data,
    input_dim,
    num_rounds=25,
    local_epochs=5,
    malicious_ids=None,
    attack='none',
    exp_name='baseline'
):
    if malicious_ids is None:
        malicious_ids = []
    
    print(f"\n{'='*60}")
    print(f"Experiment: {exp_name}")
    print(f"Attack: {attack}, Malicious: {malicious_ids}")
    print(f"{'='*60}")
    
    t0 = time.time()
    history = []
    
    # Server
    server = Server(input_dim)
    msize = model_size(server.model)
    total_comm = 0
    
    # Clients
    clients = []
    for i, d in enumerate(client_data):
        mal = i in malicious_ids
        c = Client(i, d['X'], d['y'], input_dim, malicious=mal, attack=attack)
        clients.append(c)
        if mal:
            print(f"  Client {i}: MALICIOUS ({attack})")
    
    # Global test data
    X_all = np.concatenate([d['X'] for d in client_data])
    y_all = np.concatenate([d['y'] for d in client_data])
    
    # FL Loop
    for r in range(1, num_rounds + 1):
        # Distribute
        gw = server.get_weights()
        for c in clients:
            c.set_weights(copy.deepcopy(gw))
        
        # Train
        cw_list = []
        cs_list = []
        for c in clients:
            c.train(epochs=local_epochs)
            cw_list.append(c.get_weights())
            cs_list.append(c.n_samples)
        
        # Aggregate
        server.aggregate(cw_list, cs_list)
        
        # Comm
        total_comm += msize * len(clients) * 2
        
        # Eval
        pred, loss = server.evaluate(X_all, y_all)
        acc = accuracy_score(y_all, pred)
        f1 = f1_score(y_all, pred)
        
        history.append({'round': r, 'accuracy': acc, 'f1_score': f1, 'loss': loss})
        
        if r % 5 == 0 or r == num_rounds:
            print(f"  Round {r:2d}/{num_rounds} - Acc: {acc:.4f}, F1: {f1:.4f}")
    
    # Final
    pred, loss = server.evaluate(X_all, y_all)
    final = calc_metrics(y_all, pred, loss)
    
    print(f"\nFinal: Acc={final.accuracy:.4f}, F1={final.f1_score:.4f}, Prec={final.precision:.4f}, Rec={final.recall:.4f}")
    
    return ExpResult(name=exp_name, metrics=final, time=time.time()-t0, comm_bytes=total_comm, history=history)

## 7. Run Experiments

In [ ]:
# Config
NUM_CLIENTS = 10
NUM_ROUNDS = 25
LOCAL_EPOCHS = 5
MALICIOUS_FRACTIONS = [0.1, 0.2, 0.3]

# Partition data IID
np.random.seed(42)
indices = np.random.permutation(len(y))
X_shuffled = X_normalized[indices]
y_shuffled = y[indices]

splits = np.array_split(np.arange(len(y)), NUM_CLIENTS)
client_data = [{'X': X_shuffled[s], 'y': y_shuffled[s]} for s in splits]

input_dim = X_normalized.shape[1]

print(f"Clients: {NUM_CLIENTS}")
print(f"Samples per client: ~{len(y)//NUM_CLIENTS}")
print(f"Features: {input_dim}")

In [ ]:
results = {}

# BASELINE
tf.random.set_seed(42)
np.random.seed(42)
results['baseline'] = run_experiment(
    client_data, input_dim, NUM_ROUNDS, LOCAL_EPOCHS,
    malicious_ids=[], attack='none', exp_name='baseline'
)

In [ ]:
# MODEL POISONING
print("\n" + "="*70)
print("MODEL POISONING (Sign-Flip Attack)")
print("="*70)

for frac in MALICIOUS_FRACTIONS:
    n_mal = int(NUM_CLIENTS * frac)
    mal_ids = list(range(n_mal))
    
    tf.random.set_seed(42)
    np.random.seed(42)
    name = f'model_poison_{int(frac*100)}pct'
    results[name] = run_experiment(
        client_data, input_dim, NUM_ROUNDS, LOCAL_EPOCHS,
        malicious_ids=mal_ids, attack='model', exp_name=name
    )

In [ ]:
# DATA POISONING
print("\n" + "="*70)
print("DATA POISONING (Label Flipping)")
print("="*70)

for frac in MALICIOUS_FRACTIONS:
    n_mal = int(NUM_CLIENTS * frac)
    mal_ids = list(range(n_mal))
    
    tf.random.set_seed(42)
    np.random.seed(42)
    name = f'data_poison_{int(frac*100)}pct'
    results[name] = run_experiment(
        client_data, input_dim, NUM_ROUNDS, LOCAL_EPOCHS,
        malicious_ids=mal_ids, attack='data', exp_name=name
    )

## 8. Results

In [ ]:
print("\n" + "="*100)
print("RESULTS SUMMARY")
print("="*100)

print(f"\n{'Experiment':<25} {'Accuracy':>12} {'F1 Score':>12} {'Precision':>12} {'Recall':>12} {'Loss':>12}")
print("-"*100)

for name, r in results.items():
    m = r.metrics
    print(f"{name:<25} {m.accuracy:>12.4f} {m.f1_score:>12.4f} {m.precision:>12.4f} {m.recall:>12.4f} {m.loss:>12.4f}")

base = results['baseline'].metrics
print("\n" + "-"*100)
print("DROPS FROM BASELINE")
print("-"*100)
print(f"\n{'Experiment':<25} {'Acc Drop':>12} {'F1 Drop':>12} {'Prec Drop':>12} {'Rec Drop':>12}")
print("-"*100)

for name, r in results.items():
    if name == 'baseline':
        continue
    m = r.metrics
    print(f"{name:<25} {base.accuracy-m.accuracy:>+12.4f} {base.f1_score-m.f1_score:>+12.4f} {base.precision-m.precision:>+12.4f} {base.recall-m.recall:>+12.4f}")

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

names = list(results.keys())
x = np.arange(len(names))
colors = ['green' if 'baseline' in n else 'orange' if 'model' in n else 'red' for n in names]

for ax, (metric, title) in zip(axes.flat, [
    ('accuracy', 'Accuracy'), ('f1_score', 'F1 Score'),
    ('precision', 'Precision'), ('recall', 'Recall')
]):
    vals = [getattr(results[n].metrics, metric) for n in names]
    bars = ax.bar(x, vals, color=colors)
    ax.set_title(title, fontsize=12)
    ax.set_ylabel(title)
    ax.set_xticks(x)
    ax.set_xticklabels([n.replace('_', '\n') for n in names], rotation=45, ha='right', fontsize=8)
    ax.axhline(y=getattr(base, metric), color='green', linestyle='--', alpha=0.7)
    ax.set_ylim(0, 1)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f'{v:.2f}', ha='center', fontsize=8)

plt.suptitle('TensorFlow FL: Poisoning Attack Impact', fontsize=14)
plt.tight_layout()
plt.savefig('tff_poisoning_results.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for name, r in results.items():
    rounds = [h['round'] for h in r.history]
    accs = [h['accuracy'] for h in r.history]
    f1s = [h['f1_score'] for h in r.history]
    
    ls = '-' if 'baseline' in name else '--' if 'model' in name else ':'
    lw = 2.5 if 'baseline' in name else 1.5
    axes[0].plot(rounds, accs, label=name, linestyle=ls, linewidth=lw)
    axes[1].plot(rounds, f1s, label=name, linestyle=ls, linewidth=lw)

axes[0].set_xlabel('Round')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy Over Rounds')
axes[0].legend(fontsize=8)
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Round')
axes[1].set_ylabel('F1 Score')
axes[1].set_title('F1 Score Over Rounds')
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.3)

plt.suptitle('Training Progression', fontsize=14)
plt.tight_layout()
plt.savefig('tff_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Export

In [ ]:
export = {
    'framework': 'TensorFlow Federated',
    'config': {
        'num_clients': NUM_CLIENTS,
        'num_rounds': NUM_ROUNDS,
        'local_epochs': LOCAL_EPOCHS,
        'malicious_fractions': MALICIOUS_FRACTIONS
    },
    'experiments': {n: r.to_dict() for n, r in results.items()},
    'baseline_metrics': results['baseline'].metrics.to_dict(),
    'communication': {
        'total_bytes': results['baseline'].comm_bytes,
        'per_round_bytes': results['baseline'].comm_bytes / NUM_ROUNDS
    },
    'poisoning_analysis': {
        'model_poisoning': {},
        'data_poisoning': {}
    }
}

for name, r in results.items():
    if 'model_poison' in name:
        pct = name.split('_')[-1]
        export['poisoning_analysis']['model_poisoning'][pct] = {
            'metrics_after': r.metrics.to_dict(),
            'accuracy_drop': base.accuracy - r.metrics.accuracy,
            'f1_drop': base.f1_score - r.metrics.f1_score,
            'precision_drop': base.precision - r.metrics.precision,
            'recall_drop': base.recall - r.metrics.recall,
            'loss_increase': r.metrics.loss - base.loss
        }
    elif 'data_poison' in name:
        pct = name.split('_')[-1]
        export['poisoning_analysis']['data_poisoning'][pct] = {
            'metrics_after': r.metrics.to_dict(),
            'accuracy_drop': base.accuracy - r.metrics.accuracy,
            'f1_drop': base.f1_score - r.metrics.f1_score,
            'precision_drop': base.precision - r.metrics.precision,
            'recall_drop': base.recall - r.metrics.recall,
            'loss_increase': r.metrics.loss - base.loss
        }

with open('tff_results.json', 'w') as f:
    json.dump(export, f, indent=2)

print("Saved tff_results.json")

In [ ]:
from google.colab import files
files.download('tff_results.json')
files.download('tff_poisoning_results.png')
files.download('tff_training_curves.png')
print("Downloads complete!")